In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
import nibabel as nib
import numpy as np
import glob
import csv
import tensorflow as tf
import argparse
import gc
import random
from tensorflow.keras.layers import Input, Conv3D, Conv3DTranspose, MaxPooling3D, UpSampling3D, Cropping3D, Flatten, Reshape, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import KFold, StratifiedKFold

RootFolder = "G:/File" # Modify the Root Folder
DataFolder = os.path.join(RootFolder, "DataSet/Outputs/cpac/filt_global")
PhenoTypePos = os.path.join(RootFolder, "DataSet/Phenotypic_V1_0b_preprocessed1.csv")
NewDataFolder = os.path.join(RootFolder, "NewDataSet")

FileMapping = {'func_preproc' : '_func_preproc.nii.gz',
               'rois_aal' : '_rois_aal.1D',
               'rois_cc200' : '_rois_cc200.1D',
               'rois_cc400' : '_rois_cc400.1D'}

In [2]:
def GetSubjectInfo(SubjectIDs, InfoName):
    VarDict = {}
    with open(PhenoTypePos) as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row['SUB_ID'] in SubjectIDs:
                VarDict[row['SUB_ID']] = row[InfoName]
    return VarDict

def GetASDLabels(SubjectIDs):
    DXDict = GetSubjectInfo(SubjectIDs, 'DX_GROUP')
    Labels = np.zeros(len(SubjectIDs))
    for i in range(len(SubjectIDs)):
        Labels[i] =  DXDict[SubjectIDs[i]]
    for i in range(len(Labels)):
        if Labels[i] > 0:
            Labels[i] = Labels[i] - 1
    return Labels

def GetASDSite(SubjectIDs):
    SiteId = GetSubjectInfo(SubjectIDs, 'SITE_ID')
    SiteAll = []
    for i in range(len(SubjectIDs)):
        SiteAll.append(SiteId[SubjectIDs[i]])
    return np.array(SiteAll)

In [3]:
def CaculatingCorr(A, B):
    # print(A.shape, " ", B.shape)
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)
    # print(ssA[:, None].shape, " ", ssB[None].shape)
    div = np.dot(ssA[:, None], ssB[None])
    # print(div.shape)
    div[div == 0.] = 1

    Output = np.dot(A_mA, B_mB.T) / np.sqrt(div)

    return Output

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [value]))
def _float_feature(value):
    return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))
def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))

def StoreCorrelation(ID, SubjectID, Size, Img, Type):
    SavePos = '/File/NewDataSet/Correlation/' + Type
    # print(SavePos)
    if not os.path.exists(SavePos):
        os.mkdir(SavePos)
    SaveFile = os.path.join(SavePos, SubjectID + '.tfrecords')
    if not os.path.exists(SaveFile):
        os.chdir(os.path.join(DataFolder, Type))
        RoisPos = glob.glob("*" + SubjectID + FileMapping[Type])[0]
        Rois = np.loadtxt(RoisPos, skiprows = 0)
        Corr = np.float32(CaculatingCorr(Img.reshape(-1, Img.shape[3]), Rois.T))
        Corr[np.isnan(Corr)] = 0.

        features = {}
        writer = tf.io.TFRecordWriter(SaveFile)
        feature = {
            "array" : _bytes_feature(tf.io.serialize_tensor(Corr.reshape(Size)).numpy()),
            "label" : _float_feature(Labels[ID])
        }
        tf_example = tf.train.Example(features = tf.train.Features(feature = feature))
        writer.write(tf_example.SerializeToString())
        # Haha = tf.io.parse_single_example(example.SerializeToString(), feature_decode)
        # data = tf.io.decode_raw(Haha['AAL'], out_type = tf.float32)
        # size = Haha['shape']
        writer.close()
    # print('Finishing ' + Type)

def CreateCorrelation(SubjectIDs, Labels):
    for i in range(len(SubjectIDs)):
        id = SubjectIDs[i]
        
        SavePos = os.path.join(os.path.join(NewDataFolder, "Correlation"))
        print("Searching Data", id)
        if not os.path.exists(SavePos):
            os.mkdir(SavePos)

        os.chdir(os.path.join(DataFolder, "func_preproc"))
        SubjectPos = glob.glob("*" + id + FileMapping['func_preproc'])[0]
        Subject = nib.load(SubjectPos)
        Img = Subject.get_fdata()
        
        StoreCorrelation(i, id, [61, 73, 61, 116], Img, 'rois_aal')
        # StoreCorrelation(i, id, [61, 73, 61, 200], Img, 'rois_cc200')
        # StoreCorrelation(i, id, [61, 73, 61, 392], Img, 'rois_cc400')

decode_description = {
    "array" : tf.io.FixedLenFeature([], tf.string),
    "label" : tf.io.FixedLenFeature([], tf.float32)
}

def decode_function(example_proto):
    features = tf.io.parse_single_example(example_proto, decode_description)
    features["array"] = tf.io.parse_tensor(features["array"], "float32")
    features["label"] = features["label"].numpy()
    return features

def ReadFile(IdList, Type):
    # print(IdList)
    os.chdir(os.path.join(os.path.join(NewDataFolder, "Correlation"), Type))
    Raw = tf.data.TFRecordDataset(IdList)
    X = []
    Y = []
    for features in Raw:
        Transfer = decode_function(features)
        X.append(Transfer['array'])
        Y.append(Transfer['label'])
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

# TrainId = os.path.join(RootFolder, "DataSet/ABIDEI.txt")
# SubjectIDs = np.genfromtxt(TrainId, dtype=str)
# Labels = GetASDLabels(SubjectIDs)
# CreateCorrelation(SubjectIDs, Labels)

# X, Y = ReadFile(['50004.tfrecords', '50009.tfrecords'], 'rois_cc200')
# print(X.shape)
# print(Y.shape)

In [12]:
TrainId = os.path.join(RootFolder, "DataSet/ABIDEI.txt")
SubjectIDs = np.genfromtxt(TrainId, dtype=str)
print(len(SubjectIDs))
A = random.sample(list(SubjectIDs), len(SubjectIDs) // 5)
A.sort()
File = open(os.path.join(RootFolder, "DataSet/ABIDEII.txt"), mode = "w")
for item in A:
    File.write(item + '\n')
File.close()
# np.savetxt(os.path.join(RootFolder, "DataSet/ABIDEII.txt"), A)
# 
# File.write(str(A))
# File.close()
# A

884


In [4]:
def AutoEncoderModel(input_size):
    N = input_size[3]
    origin = Input(shape = input_size)
    x = Conv3D(16, (3, 3, 3), activation = 'relu', padding = 'same')(origin)
    x = MaxPooling3D((2, 2, 2), padding = 'same')(x)
    x = Conv3D(8, (3, 3, 3), activation = 'relu', padding = 'same')(x)
    x = MaxPooling3D((2, 2, 2), padding = 'same')(x)
    x = Conv3D(4, (3, 3, 3), activation = 'relu', padding = 'same')(x)
    x = MaxPooling3D((2, 2, 2), padding = 'same')(x)
    x = Flatten()(x)
    encoded = Dense(64, activation = 'relu')(x)

    x = Dense(8 * 10 * 8 * 4, activation = 'relu')(encoded)
    x = Reshape([8, 10, 8, 4])(x)
    x = Conv3D(4, (3, 3, 3), activation = 'relu', padding = 'same')(x)
    x = UpSampling3D((2, 2, 2))(x)
    x = Conv3D(8, (3, 3, 3), activation = 'relu', padding = 'same')(x)
    x = UpSampling3D((2, 2, 2))(x)
    x = Conv3D(16, (3, 3, 3), activation = 'relu', padding = 'same')(x)
    x = UpSampling3D((2, 2, 2))(x)
    x = Conv3D(N, (3, 3, 3), activation = 'relu', padding = 'same')(x)
    decoded = Cropping3D(((3, 0), (7, 0), (3, 0)))(x)

    autoencoder = Model(origin, decoded)
    encoder = Model(origin, encoded)
    # autoencoder.summary()
    return encoder, autoencoder

AutoEncoderModel((61, 73, 61, 116))

(<keras.engine.functional.Functional at 0x22abeac2910>,
 <keras.engine.functional.Functional at 0x22abeacfb80>)

In [39]:
def Generator(SubjectIDs, Id, batchsz):
    random.shuffle(Id)
    NumOfBatch = (len(Id) // batchsz) + (len(Id) % batchsz != 0)
    # print(NumOfBatch, len(TrainId), batchsz)
    n = 1
    # print("Here")
    while 1:
        Ids = []
        X = []
        Y = []
        if n > NumOfBatch:
            n = 1
        if n == NumOfBatch:
            for i in range((n - 1) * batchsz, len(Id)):
                Ids.append(str(SubjectIDs[Id[i]]) + '.tfrecords')
            X, Y = ReadFile(IdList = Ids, Type = 'rois_aal')
        else:
            for i in range((n - 1) * batchsz, n * batchsz):
                Ids.append(str(SubjectIDs[Id[i]]) + '.tfrecords')
            X, Y = ReadFile(IdList = Ids, Type = 'rois_aal')
        n = n + 1
        gc.collect()
        yield X, X

# def TrainAndEvaluate(SubjectIDs, TrainId, TestId, params):
#     batchsz = params['batch_size']
#     modelE, modelAE = AutoEncoderModel((61, 73, 61, 116))
#     modelAE.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
#     Ids = []
#     for i in range(len(TrainId)):
#         Ids.append(str(SubjectIDs[TrainId[i]]) + '/aal.tfrecords')
#     TrainX, TrainY = ReadFile(IdList = Ids)
#     history = modelAE.fit(TrainX, TrainX, epochs = params['epochs'], batch_size = batchsz, verbose = 1, validation_split = 0.2)
#     Ids = []
#     for i in range(len(TestId)):
#         Ids.append(str(SubjectIDs[TestId[i]]) + '/aal.tfrecords')
#     TestX, TestY = ReadFile(IdList = Ids)
#     loss, accuracy = modelAE.evaluate(TestX, TestX)
#     return loss, history, modelE

def TrainAndEvaluate(SubjectIDs, TrainId, TestId, params):
    batchsz = params['batch_size']
    modelE, modelAE = AutoEncoderModel((61, 73, 61, 116))
    modelE.compile(optimizer = 'adam', loss = 'mse')
    modelAE.compile(optimizer = 'adam', loss = 'mse')

    history = modelAE.fit(Generator(SubjectIDs, TrainId, batchsz), steps_per_epoch = (len(TrainId) // batchsz), epochs = params['epochs'], verbose = 1)

    scores = modelAE.evaluate(Generator(SubjectIDs, TestId, batchsz), steps = (len(TestId) // batchsz))

    return scores, modelE

def AutoEncoderResult(SubjectIDs, params):
    Loss = []
    for i, (train, test) in enumerate(params['skf']):
        print('Start Train ' + str(i + 1) + ' Fold')
        gc.collect()
        loss, history, modelE = TrainAndEvaluate(SubjectIDs, train, test, params)
        Loss.append(loss)
    return Loss

def GetAEModel(SubjectIDs, params):
    train, test = params['skf']
    scores, modelE = TrainAndEvaluate(SubjectIDs, train, test, params)
    return scores, modelE

In [40]:
parser = argparse.ArgumentParser(description = 'Autoencoder')
parser.add_argument('-l', default = 0.001, type = float, help = 'Learning rate')
parser.add_argument('-e', default = 50, type = int, help = 'epochs')
parser.add_argument('-b', default = 4, type = int, help = 'batch_size')
parser.add_argument('-g', default = 1, type = int, help = 'Number of GPUs to use during the training')
args = parser.parse_args(args = [])
params = dict()
params['lrate'] = args.l
params['epochs'] = args.e
params['batch_size'] = args.b
params['gpu_count'] = args.g

TrainPos = os.path.join(RootFolder, "DataSet/ABIDEII.txt")
SubjectIDs = np.genfromtxt(TrainPos, dtype=str)
Labels = GetASDLabels(SubjectIDs)
TestX = random.sample(range(0, len(SubjectIDs)), len(SubjectIDs) // 5)
TrainX = list(np.array([i for i in range(len(SubjectIDs)) if i not in TestX]))
# print(TestX)
# print(TrainX)
params['skf'] = (TrainX, TestX)
scores, modelE = GetAEModel(SubjectIDs, params)
# Site = GetASDSite(SubjectIDs)
# skf = KFold(n_splits = 10, shuffle = True).split(Site)
# params['skf'] = skf
# history = AutoEncoderResult(SubjectIDs, params)
# CreateCorrelation(SubjectIDs, Labels)

Epoch 1/50
35/35 [==============================] - 201s 5s/step - loss: 0.0087
Epoch 2/50
35/35 [==============================] - 186s 5s/step - loss: 0.0087
Epoch 3/50
35/35 [==============================] - 186s 5s/step - loss: 0.0085
Epoch 4/50
35/35 [==============================] - 187s 5s/step - loss: 0.0085
Epoch 5/50
35/35 [==============================] - 187s 5s/step - loss: 0.0085
Epoch 6/50
35/35 [==============================] - 189s 5s/step - loss: 0.0083
Epoch 7/50
35/35 [==============================] - 189s 5s/step - loss: 0.0084
Epoch 8/50
35/35 [==============================] - 189s 5s/step - loss: 0.0084
Epoch 9/50
35/35 [==============================] - 192s 5s/step - loss: 0.0083
Epoch 10/50
35/35 [==============================] - 190s 5s/step - loss: 0.0083
Epoch 11/50
35/35 [==============================] - 188s 5s/step - loss: 0.0083
Epoch 12/50
35/35 [==============================] - 188s 5s/step - loss: 0.0082
Epoch 13/50
35/35 [==================

In [41]:
A = modelE.get_weights()
# print(A)
modelE.save(os.path.join(RootFolder, 'my_model.h5'))

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
TrainPos = os.path.join(RootFolder, "DataSet/ABIDEI.txt")
SubjectIDs = np.genfromtxt(TrainPos, dtype=str)
Labels = GetASDLabels(SubjectIDs)
feature = []
target = []
modelE = load_model(os.path.join(RootFolder, 'my_model.h5'))
for i in range(len(SubjectIDs)):
    Ids = [str(SubjectIDs[i]) + '.tfrecords']
    # print(Ids)
    X, Y = ReadFile(IdList = Ids, Type = 'rois_aal')
    X = modelE.predict(X)
    feature.append(X.reshape([64]))
    target.append(Y)
print(np.array(feature).shape)
print(np.array(target).shape)


(884, 64)
(884, 1)


In [5]:
feature = np.array(feature)
target = np.array(target).reshape(len(target))
np.savez(os.path.join(RootFolder, "final.npz"), x = feature, y = target)

In [119]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
data = np.load(os.path.join(RootFolder, "final.npz"))
x_train, x_test, y_train, y_test = train_test_split(data['x'], data['y'], test_size=0.2)
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(x_train, y_train)
print(knn.score(x_train, y_train))
print(knn.score(x_test, y_test))

0.6251768033946252
0.5536723163841808


In [68]:
from sklearn import svm
data = np.load(os.path.join(RootFolder, "final.npz"))
x_train, x_test, y_train, y_test = train_test_split(data['x'], data['y'], test_size=0.2)
clf = svm.SVC(C = 0.5, kernel = 'rbf', gamma = 1, decision_function_shape = 'ovr')
clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))
print(clf.score(x_test, y_test))

0.5912305516265912
0.6214689265536724


In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
data = np.load(os.path.join(RootFolder, "final.npz"))
x_train, x_test, y_train, y_test = train_test_split(data['x'], data['y'], test_size=0.2)
y_train.reshape((-1, 1))
# print(y_train.shape)
# print(y_train)
def Models(input_size):
    origin = Input(shape = input_size)
    x = Dense(1000, activation = 'relu')(origin)
    x = Dense(500, activation = 'relu')(x)
    result = Dense(1, activation = 'sigmoid')(x)

    model = Model(origin, result)
    # autoencoder.summary()
    return model

model = Models(64)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs = 100, verbose = 1)
scores = model.evaluate(x_test, y_test)

Epoch 1/100
23/23 [==============================] - 1s 14ms/step - loss: 0.6953 - accuracy: 0.5205
Epoch 2/100
23/23 [==============================] - 0s 10ms/step - loss: 0.6936 - accuracy: 0.5431
Epoch 3/100
23/23 [==============================] - 0s 7ms/step - loss: 0.6858 - accuracy: 0.5629
Epoch 4/100
23/23 [==============================] - 0s 6ms/step - loss: 0.6810 - accuracy: 0.5587
Epoch 5/100
23/23 [==============================] - 0s 7ms/step - loss: 0.6762 - accuracy: 0.5672
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 0.6759 - accuracy: 0.5644
Epoch 7/100
23/23 [==============================] - 0s 6ms/step - loss: 0.6803 - accuracy: 0.5446
Epoch 8/100
23/23 [==============================] - 0s 6ms/step - loss: 0.6744 - accuracy: 0.5601
Epoch 9/100
23/23 [==============================] - 0s 6ms/step - loss: 0.6651 - accuracy: 0.5926
Epoch 10/100
23/23 [==============================] - 0s 7ms/step - loss: 0.6617 - accuracy: 0.5856
Epoch 1